### Model Training

In [1]:
import csv
import pandas as pd

In [2]:
try:
    import models
except ImportError:
    print('Please upload models.py to you current directory')
    raise

2024-04-21 19:25:33.627074: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-21 19:25:33.678202: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-21 19:25:34.470651: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
df = pd.read_pickle("./data/cve")

In [4]:
train = df[:120000]
test = df[120000:]

In [5]:
from models import BERTmodel

In [6]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    
    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: NVIDIA A100 80GB PCIe


In [19]:
metric = 'privileges_required'

In [9]:
pr_model = BERTmodel('privileges_required', train, test)

In [10]:
pr_model.train(epochs=2)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i


======== Epoch 1 / 2 ========
Training...
  Batch 2,000  of  13,500.    Elapsed: 0:13:25.
  Batch 4,000  of  13,500.    Elapsed: 0:26:49.
  Batch 6,000  of  13,500.    Elapsed: 0:40:13.
  Batch 8,000  of  13,500.    Elapsed: 0:53:37.
  Batch 10,000  of  13,500.    Elapsed: 1:07:02.
  Batch 12,000  of  13,500.    Elapsed: 1:20:31.

  Average training loss: 0.46
  Training epoch took: 1:30:33

Running Validation...
  Accuracy: 0.86
  Validation Loss: 0.41
  Validation took: 0:03:16

======== Epoch 2 / 2 ========
Training...
  Batch 2,000  of  13,500.    Elapsed: 0:13:25.
  Batch 4,000  of  13,500.    Elapsed: 0:42:40.
  Batch 6,000  of  13,500.    Elapsed: 1:16:15.
  Batch 8,000  of  13,500.    Elapsed: 1:49:49.
  Batch 10,000  of  13,500.    Elapsed: 2:23:24.
  Batch 12,000  of  13,500.    Elapsed: 2:57:00.

  Average training loss: 0.36
  Training epoch took: 3:22:06

Running Validation...
  Accuracy: 0.87
  Validation Loss: 0.45
  Validation took: 0:08:16

Training complete!
Total tr

,Training Loss,Valid. Loss,Valid. Accur.,Training Time,Validation Time
epoch,,,,,
1,0.46,0.41,0.86,1:30:33,0:03:16
2,0.36,0.45,0.87,3:22:06,0:08:16


In [11]:
pr_model.test()

0.8106806950422445

In [12]:
print(pr_model.getClassificationStats())

{'accuracy': 0.8106806950422445, 'precision': 0.8106806950422445, 'recall': 0.8106806950422445, 'f1': 0.8106806950422445, 'mcc': 0.6561289979997119, 'cm': array([[15854,  1292,    92],
       [ 3008,  7818,   223],
       [  657,   666,  1755]])}


In [13]:
pr_model.saveModel()
pr_model.savePredictions()

### Push model to huggingface hub

In [14]:
from huggingface_hub import notebook_login

In [16]:
notebook_login()

In [17]:
from transformers import BertForSequenceClassification

In [20]:
pr_model = BertForSequenceClassification.from_pretrained(f"./{metric}_model/")

In [21]:
pr_model.push_to_hub(f"Vijaybhk/{metric.title()}-BERT")

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Vijaybhk/Privileges_Required-BERT/commit/50f501d12b7ccd51791a46dcb88804e9ec460c1c', commit_message='Upload BertForSequenceClassification', commit_description='', oid='50f501d12b7ccd51791a46dcb88804e9ec460c1c', pr_url=None, pr_revision=None, pr_num=None)